# RO47019: Intelligent Control Systems Practical Assignment
* Period: 2022-2023, Q3
* Course homepage: https://brightspace.tudelft.nl/d2l/home/500969
* Instructor: Cosimo Della Santina (C.DellaSantina@tudelft.nl)
* Teaching assistant: Ruben Martin Rodriguez (R.MartinRodriguez@student.tudelft.nl)
* (c) TU Delft, 2023

Make sure you fill in any place that says `YOUR CODE HERE` or `YOUR ANSWER HERE`. Remove `raise NotImplementedError()` afterwards. Moreover, if you see an empty cell, please DO NOT delete it, instead run that cell as you would run all other cells. Please fill in your name(s) and other required details below:

In [ ]:
# Please fill in your names, student numbers, netID, and emails below.
STUDENT_1_NAME = ""
STUDENT_1_STUDENT_NUMBER = ""
STUDENT_1_NETID = ""
STUDENT_1_EMAIL = ""

In [ ]:
# Note: this block is a check that you have filled in the above information.
# It will throw an AssertionError until all fields are filled
assert STUDENT_1_NAME != ""
assert STUDENT_1_STUDENT_NUMBER != ""
assert STUDENT_1_NETID != ""
assert STUDENT_1_EMAIL != ""

### General announcements

* Do *not* share your solutions, and do *not* copy solutions from others. By submitting your solutions, you claim that you alone are responsible for this code.

* Do *not* email questions directly, since we want to provide everybody with the same information and avoid repeating the same answers. Instead, please post your questions regarding this assignment in the correct support forum on Brightspace, this way everybody can benefit from the response. If you do have a particular question that you want to ask directly, please use the scheduled Q&A hours to ask the TA.

* There is a strict deadline for each assignment. Students are responsible to ensure that they have uploaded their work in time. So, please double check that your upload succeeded to the Brightspace and avoid any late penalties.

* This [Jupyter notebook](https://jupyter.org/) uses `nbgrader` to help us with automated tests. `nbgrader` will make various cells in this notebook "uneditable" or "unremovable" and gives them a special id in the cell metadata. This way, when we run our checks, the system will check the existence of the cell ids and verify the number of points and which checks must be run. While there are ways that you can edit the metadata and work around the restrictions to delete or modify these special cells, you should not do that since then our nbgrader backend will not be able to parse your notebook and give you points for the assignment. You are free to add additional cells, but if you find a cell that you cannot modify or remove, please know that this is on purpose.

* This notebook will have in various places a line that throws a `NotImplementedError` exception. These are locations where the assignment requires you to adapt the code! These lines are just there as a reminder for youthat you have not yet adapted that particular piece of code, especially when you execute all the cells. Once your solution code replaced these lines, it should accordingly *not* throw any exceptions anymore.

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

# Task 1.0 - Generate the datasets (0p)

**Authors:** Tomás Coleman (T.Coleman@tudelft.nl), Chuhan Zhang (C.Zhang-8@tudelft.nl)

### NOTE: This notebook is a replacement for the CNN-part in `task_1a_generate_data.ipynb`

Due to the memory limitation of GitHub Codespaces, we recommend using this notebook instead to generate the **CNN** data. The procedure of generating data is exactly the same as the original notebook.

### Training data for tasks 1.1-1.4

In order to train the model, we require data. Therefore, we generate images of the robot at various positions in its state space. 
At every position chosen in the state space, an RGB image with dimensions 32x32x3, is generated as observation data $x$ together with the corresponding robot state $s=(\theta, \dot{\theta})$, where $\theta = [\theta_1, \theta_2]$ are the link angles of the robot, and $\dot{\theta} = [\dot{\theta_1}, \dot{\theta_2}]$ are the link angular velocities of the robot. 
The angle $\theta_1$ of link 1 and $\theta_2$ of link 2 are both defined with respect to the right horizontal position (or x-axis), both are wrapped to the $[-\pi, \pi]$ domain. 

We also generate a test set, which are observations of the robot sampled at every $2^{\circ}$, of the first link and at every $2^{\circ}$ of the second link for every sample of the first, giving 32400 (180x180) observations. 

Run the cells below to generate the data needed for the problems in `task_1b_train_NN.ipynb`.

**This notebook is not graded but is required for all tasks of problem 1.**

**Please do __not__ include the datasets of Problem 1 in your final submission!** I.e. exclude the `source/problem_1/datasets` folder from your ZIP archive.

In [ ]:
# Reloads the python files outside of this notebook automatically
%load_ext autoreload
%autoreload 2

# import all Python modules
from distutils.util import strtobool
from jax.config import config as jax_config

jax_config.update("jax_platform_name", "cpu")  # set default device to 'cpu'
jax_config.update("jax_enable_x64", True)  # double precision
from jax import jit, lax, random
from jax import numpy as jnp
import numpy as onp
import matplotlib.pyplot as plt
import os
from pathlib import Path
from typing import Dict
import shutil
import torch

from jax_double_pendulum.robot_parameters import ROBOT_PARAMS
from jax_double_pendulum.kinematics import forward_kinematics
from utils import *


# define boolean to check if the notebook is run for the purposes of autograding
AUTOGRADING = strtobool(os.environ.get("AUTOGRADING", "false"))

# folder to save the dataset to
datasets_folder = Path("datasets")
datasets_folder.mkdir(exist_ok=True, parents=True)

# create directory for plots
outputs_dir = Path("outputs")
outputs_dir.mkdir(parents=True, exist_ok=True)

## Generating dataset of state images 

### Training data for tasks 1.1-1.4

In order to train the model, we require data. Therefore, we generate images of the robot at various positions in its state space. 
At every position chosen in the state space, an RGB image with dimensions 32x32x3, is generated as observation data $x$ together with the corresponding robot state $s=(\theta, \dot{\theta})$, where $\theta = [\theta_1, \theta_2]$ are the link angles of the robot, and $\dot{\theta} = [\dot{\theta_1}, \dot{\theta_2}]$ are the link angular velocities of the robot. 
The angle $\theta_1$ of link 1 and $\theta_2$ of link 2 are both defined with respect to the right horizontal position (or x-axis), both are wrapped to the $[-\pi, \pi]$ domain. 

#### Training data

The training data constists of 20,000 images of the robot with link angles that are randomly sampled from the state space. 
- The link angles are first sampled and saved into the dataset with the label `th_curr_ss` to be used later as the ground truth labels. 
- From these angles, the robot is rendered into the 32x32x3 RGB images. Link 1 is blue and link 2 is red.

#### Test data
We also generate a test set, which are again 32x32x3 image observations of the robot sampled at every $2^{\circ}$, of the first link and at every $2^{\circ}$ of the second link for every sample of the first, giving 32400 (180x180) observations. Having a test data set larger than the training set is unusual in practice as we want our model to have as much data to train on in practice. As this is a training excercise (for you and the neural networks :), we generate this large test set over the state space so you can fully analyse the performance of the trained Neural Networks.

Run the cells below to generate the data needed for the problems in `task_1b_train_NN.ipynb`.

In [ ]:
@jit
def save_test_data_to_dataset(
    _dataset: Dict[str, jnp.ndarray],
    initial_conditions: jnp.array,
) -> Dict[str, jnp.ndarray]:
    """
    puts the robot angle and x-y joint position in the dataset
    """
    num_ic = initial_conditions.shape[0]
    _dataset["th_curr_ss"] = _dataset["th_curr_ss"].at[:].set(initial_conditions)

    # Gets the robot elbow and end effector x-y coordinates from the 'forward_kinematics' function
    def _for_loop_cart_fun(idx, _dataset: Dict):
        _x_eb, _x = forward_kinematics(ROBOT_PARAMS, _dataset["th_curr_ss"][idx])
        _dataset["x_eb_ts"] = _dataset["x_eb_ts"].at[idx].set(_x_eb)
        _dataset["x_ts"] = _dataset["x_ts"].at[idx].set(_x)
        return _dataset

    _dataset = lax.fori_loop(
        lower=0, upper=num_ic, body_fun=_for_loop_cart_fun, init_val=_dataset
    )

    return _dataset

In [ ]:
def save_image_data_to_dataset(
    _dataset: Dict[str, jnp.ndarray], ROBOT_PARAMS: Dict[str, jnp.ndarray]
) -> Dict[str, jnp.ndarray]:
    """
    This function calls the 'draw_robot' function to draw the image
    of the robot at each given theta values and returns the given
    dataset with these drawn images corresponding to given angles
    """
    _dataset = draw_robot(_dataset, ROBOT_PARAMS)

    return _dataset

In [ ]:
# Set seed so everyone has the same "random" training dataset
onp.random.seed(42)

# simulation parameters
num_samples_train = 20000
img_size = 32

# Sample the state space of -pi to pi for both robot links
training_angles_sampled = onp.random.uniform(-onp.pi, onp.pi, [num_samples_train, 2])

# Generate test data: 40 x 40 images to evenly cover the state space
th1_range = jnp.arange(-jnp.pi, jnp.pi, jnp.pi / 45.0)
th2_range = jnp.arange(-jnp.pi, jnp.pi, jnp.pi / 45.0)

test_angles = jnp.array(jnp.meshgrid(th1_range, th2_range)).T.reshape(-1, 2)

num_samples_test = len(th1_range) * len(th2_range)

# initialise the train and test datasets with the appropriate sized arrays
# for the link angles "th_curr_ss", x-y elbow position "x_eb_ts",
# x-y end effector position "x_ts" and the rendered image of the
# robot at "th_curr_ss"
training_dataset = {
    "th_curr_ss": jnp.zeros((num_samples_train, 2)),
    "x_eb_ts": jnp.zeros((num_samples_train, 2)),
    "x_ts": jnp.zeros((num_samples_train, 2)),
    "th_pix_curr": jnp.zeros(
        (num_samples_train, img_size, img_size, 3), dtype=jnp.uint8
    ),
}

test_dataset = {
    "th_curr_ss": jnp.zeros((num_samples_test, 2)),
    "x_eb_ts": jnp.zeros((num_samples_test, 2)),
    "x_ts": jnp.zeros((num_samples_test, 2)),
    "th_pix_curr": jnp.zeros(
        (num_samples_test, img_size, img_size, 3), dtype=jnp.uint8
    ),
}

# input the initial values in the dataset
training_dataset = save_test_data_to_dataset(training_dataset, training_angles_sampled)
test_dataset = save_test_data_to_dataset(test_dataset, test_angles)

print("Rendering images of the robot for the training set ...")
training_dataset = save_image_data_to_dataset(training_dataset, ROBOT_PARAMS)

print("Rendering images of the robot for the test set ...")
test_dataset = save_image_data_to_dataset(test_dataset, ROBOT_PARAMS)

In [ ]:
# Now save the dataset to a file so we can access it in the notebook for tasks 1.1-1.4
jnp.savez(
    file=str(datasets_folder / "dataset_double_pendulum_train.npz"), **training_dataset
)
jnp.savez(
    file=str(datasets_folder / "dataset_double_pendulum_test.npz"), **test_dataset
)

In [ ]:
def data_distribution(_theta):
    bin_no = 20
    heatmap, xedges, yedges = onp.histogram2d(_theta[:, 0], _theta[:, 1], bins=bin_no)
    extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
    plt.clf()
    plt.imshow(heatmap.T, extent=extent)
    plt.colorbar()
    plt.xlabel("Link 1 angles (rad)")
    plt.ylabel("Link 2 angles (rad)")
    plt.title("Heat map of sample frequency in the state space")
    plt.show()

## Lets see what data we have got

Run the cells below to see the distribution of the train and test data set angles over the state space. You can see the training set has a far higher number of samples in some areas compared to others while the test set is evenly spread out. 

In [ ]:
# show the distribution of training samples in the state space
data_distribution(training_dataset["th_curr_ss"])

In [ ]:
# shows the distribution of test samplesin the state space
data_distribution(test_dataset["th_curr_ss"])

Run the cells below to show what the rendered image of the robot actually looks like

In [ ]:
# Print a random image from the training dataset
index = onp.random.randint(0, num_samples_train)
print("index", index)
print("theta: ", training_dataset["th_curr_ss"][index])
plt.imshow(training_dataset["th_pix_curr"][index, :, :, :])

In [ ]:
# Print a random image from the test dataset
index = onp.random.randint(0, 180 * 180)
print("index", index)
print("theta: ", test_dataset["th_curr_ss"][index])
plt.imshow(test_dataset["th_pix_curr"][index, :, :, :])